In [3]:
pwd

'/media/lala/로컬 디스크/python/ml_dacon_haja'

In [19]:
import os
import random
from PIL import Image
from torchvision.transforms import Compose, RandomApply, RandomRotation, RandomAffine, RandomHorizontalFlip, RandomVerticalFlip, RandomResizedCrop

# 증강된 이미지를 저장할 경로
aug_dir = './data2/train'

# train 데이터셋 폴더 경로
train_dir = './data/train'

# 라벨별 이미지 개수 카운트
label_counts = {}
for label in os.listdir(train_dir):
    label_dir = os.path.join(train_dir, label)
    if os.path.isdir(label_dir):
        image_files = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if os.path.isfile(os.path.join(label_dir, f))]
        label_counts[label] = len(image_files)

# 라벨별 이미지 개수를 500장으로 맞추기 위해 적용할 transform 정의
transform = Compose([
    RandomApply([RandomRotation((-15, 15))], p=0.5),
    RandomApply([RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=(-10, 10))], p=0.5),
    RandomApply([RandomHorizontalFlip()], p=0.5),
    RandomApply([RandomVerticalFlip()], p=0.5),
    RandomApply([RandomResizedCrop(size=224, scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333))], p=0.5),
    RandomApply([GaussianBlur(kernel_size=5)], p=0.5)
])

# 라벨별 이미지 개수가 500장 이하인 경우, transform 적용한 증강 이미지 추가
for label in label_counts.keys():
    label_dir = os.path.join(train_dir, label)
    image_files = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if os.path.isfile(os.path.join(label_dir, f))]
    
    if label_counts[label] < 300:
        # 라벨별 이미지 개수가 500장보다 작은 경우 부족한 개수만큼 이미지를 증강시켜 추가
        augment_count = 300 - label_counts[label]
        for i in range(augment_count):
            image_file = random.choice(image_files)
            image = Image.open(image_file)
            transformed_image = transform(image)
            
            # 저장할 디렉토리가 없을 경우 생성
            if not os.path.exists(os.path.join(aug_dir, label)):
                os.makedirs(os.path.join(aug_dir, label))
            
            filename, ext = os.path.splitext(os.path.basename(image_file))
            transformed_image.save(os.path.join(aug_dir, label, f'aug_{i}_{filename}{ext}'))
    else:
                # 라벨별 이미지 개수가 500장 이상인 경우, 랜덤하게 500장 선택해서 저장
        selected_files = random.sample(image_files, 300)
        for image_file in selected_files:
            image = Image.open(image_file)
            
            # 저장할 디렉토리가 없을 경우 생성
            if not os.path.exists(os.path.join(aug_dir, label)):
                os.makedirs(os.path.join(aug_dir, label))
            
            filename, ext = os.path.splitext(os.path.basename(image_file))
            image.save(os.path.join(aug_dir, label, f'aug_{filename}{ext}'))


In [22]:
sorted_data = sorted(label_counts.items(), key=lambda x: x[1])

In [23]:
sorted_data

[('반점', 3),
 ('틈새과다', 5),
 ('가구수정', 12),
 ('녹오염', 14),
 ('이음부불량', 17),
 ('울음', 22),
 ('창틀,문틀수정', 27),
 ('피스', 51),
 ('들뜸', 54),
 ('석고수정', 57),
 ('면불량', 99),
 ('몰딩수정', 130),
 ('오타공', 142),
 ('곰팡이', 145),
 ('터짐', 162),
 ('꼬임', 210),
 ('걸레받이수정', 307),
 ('오염', 595),
 ('훼손', 1405)]